In [29]:
# Automatically reload modules after executing each cell.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# General imports
import os
import tensorflow as tf
from scipy import stats

# Utility imports
from utils.losses import *
from utils.plotting import *
from utils.training import *

np.random.seed(666) # Need to do more to ensure data is the same across runs.

In [31]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # pick a number < 4 on ML4HEP; < 3 on Voltan 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Zenodo Data

In [32]:
mc = np.load("data/zenodo/Pythia21_Zjet_pTZ-200GeV_0.npz")

sim_pt =  mc['sim_jets'][:, 0]
sim_eta = mc['sim_jets'][:, 1]
sim_phi = mc['sim_jets'][:, 2]
sim_m =   mc['sim_jets'][:, 3]

sim = np.vstack([sim_pt, sim_eta, sim_phi, sim_m]).T

data = np.load("data/zenodo/Herwig_Zjet_pTZ-200GeV_0.npz")

dat_pt =  data['sim_jets'][:, 0]
dat_eta = data['sim_jets'][:, 1]
dat_phi = data['sim_jets'][:, 2]
dat_m =   data['sim_jets'][:, 3]

dat = np.vstack([dat_pt, dat_eta, dat_phi, dat_m]).T

In [33]:
X = np.vstack((sim, dat))
y = np.hstack((np.zeros(sim.shape[0]), np.ones(dat.shape[0])))

In [34]:
np.save('data/zenodo/X.npy', X)
np.save('data/zenodo/y.npy', y)

# BCE $C$ Parametrization

In [37]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':bce, 'd':4}
params_2 = {'loss':tanh_bce, 'd':4, 'output': 'linear'}
params_3 = {'loss':arctan_bce, 'd':4,  'output': 'linear'}

filestr = 'models/flows/c_bce/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
X = np.load('data/zenodo/X.npy')
y = np.load('data/zenodo/y.npy')
data, m, s = split_data(X, y)

In [38]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 

2023-01-12 09:24:50.497964: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 09:24:51.593554: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20450 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:41:00.0, compute capability: 7.5


0.6906995177268982 	 44	0.6907417178153992 	 68	0.690867006778717 	 69	
1 0.6908239722251892 	 34	0.6908103227615356 	 37	0.6906263828277588 	 48	
2 0.6907179951667786 	 51	0.6906919479370117 	 49	0.690682590007782 	 48	
3 0.6907135844230652 	 48	0.690755307674408 	 31	0.6907156109809875 	 49	
4 0.6906711459159851 	 42	0.690698504447937 	 45	0.6907040476799011 	 37	
5 0.6907603740692139 	 33	0.6907447576522827 	 41	0.690754234790802 	 39	
6 0.6907861828804016 	 52	0.6906776428222656 	 49	0.6906707882881165 	 37	
7 0.6906940340995789 	 64	0.6907763481140137 	 67	0.6906723976135254 	 53	
8 0.6906762719154358 	 55	0.6907480955123901 	 58	0.6907429695129395 	 57	
9 0.6906816363334656 	 35	0.6907457113265991 	 58	0.6907413005828857 	 69	
10 0.6905913949012756 	 50	0.6907194256782532 	 38	0.6907811164855957 	 45	
11 0.6907349228858948 	 32	0.6907764077186584 	 26	0.6906856894493103 	 45	
12 0.6907081007957458 	 49	0.6907059550285339 	 46	0.6906400918960571 	 41	
13 0.6906149983406067 	 51	0.

# MSE $C$ Parametrization

In [39]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':mse, 'd':4}
params_2 = {'loss':tanh_mse, 'd':4, 'output': 'linear'}
params_3 = {'loss':arctan_mse, 'd':4,  'output': 'linear'}

filestr = 'models/flows/c_mse/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
X = np.load('data/zenodo/X.npy')
y = np.load('data/zenodo/y.npy')
data, m, s = split_data(X, y)

In [40]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.2487918585538864 	 44	0.2488592118024826 	 62	0.24879169464111328 	 44	
1 0.24879445135593414 	 35	0.2488059252500534 	 34	0.24876494705677032 	 60	
2 0.2487405240535736 	 27	0.2487792819738388 	 62	0.24879468977451324 	 41	
3 0.24876955151557922 	 52	0.2487633228302002 	 52	0.24876849353313446 	 32	
4 0.24879872798919678 	 74	0.24880769848823547 	 41	0.24876409769058228 	 50	
5 0.24877861142158508 	 48	0.24888163805007935 	 25	0.24878698587417603 	 70	
6 0.24879713356494904 	 34	0.24878770112991333 	 53	0.24878758192062378 	 51	
7 0.24876689910888672 	 55	0.2487829029560089 	 37	0.24878902733325958 	 45	
8 0.2488153576850891 	 31	0.24877816438674927 	 29	0.24875718355178833 	 47	
9 0.24877800047397614 	 36	0.24878820776939392 	 57	0.2488320916891098 	 43	
10 0.24881359934806824 	 36	0.24876639246940613 	 56	0.24878093600273132 	 50	
11 0.248780757188797 	 40	0.24884605407714844 	 32	0.2488146424293518 	 44	
12 0.24878522753715515 	 39	0.2488068789243698 	 47	0.24878068268299103 	 

# MLC $C$ Parametrization

In [41]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'd': 4, 'output': 'relu'}
params_2 = {'loss':square_mlc, 'd': 4, 'output': 'linear'}
params_3 = {'loss':exp_mlc, 'd': 4, 'output': 'linear'}

filestr = 'models/flows/c_mlc/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
X = np.load('data/zenodo/X.npy')
y = np.load('data/zenodo/y.npy')
data, m, s = split_data(X, y)

In [42]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 -0.004747516941279173 	 100	0.5741267204284668 	 19	-0.004951126873493195 	 55	
1 -0.004961359314620495 	 96	-0.004297634121030569 	 51	-0.004879198502749205 	 62	
2 -0.004704447463154793 	 99	-0.00402824254706502 	 100	-0.004862267058342695 	 34	
3 -0.004902618005871773 	 100	-0.0011076629161834717 	 72	-0.004953908734023571 	 42	
4 -0.0048790122382342815 	 80	-0.003671456826850772 	 71	-0.005042744800448418 	 47	
5 -0.0046281591057777405 	 91	-0.00442713126540184 	 77	-0.004943911451846361 	 47	
6 -0.0051150997169315815 	 94	-0.004089852329343557 	 100	-0.005051653366535902 	 39	
7 -0.004974839743226767 	 100	-0.002185442252084613 	 49	-0.005117975641041994 	 37	
8 -0.005017031915485859 	 100	-0.0035009009297937155 	 100	-0.00495024723932147 	 27	
9 -0.005024030804634094 	 79	-0.0048811896704137325 	 100	-0.005125182680785656 	 51	
10 -0.004371485207229853 	 100	-0.0004919333732686937 	 100	-0.0048942905850708485 	 34	
11 -0.004887258168309927 	 100	-0.003719356143847108 	 59	-0.00

# SQR $C$ Parametrization

In [43]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'd': 4, 'output': 'relu'}
params_2 = {'loss':square_sqr, 'd': 4, 'output': 'linear'}
params_3 = {'loss':exp_sqr, 'd': 4, 'output': 'linear'}

filestr = 'models/flows/c_sqr/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
X = np.load('data/zenodo/X.npy')
y = np.load('data/zenodo/y.npy')
data, m, s = split_data(X, y)

In [44]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 1.0033738613128662 	 100	12.322500228881836 	 18	0.9975720047950745 	 47	
1 0.9995337724685669 	 100	24.945615768432617 	 19	0.99754399061203 	 49	
2 0.9992356896400452 	 100	1.4287983179092407 	 51	0.9975860118865967 	 50	
3 1.0035794973373413 	 100	1.0069661140441895 	 100	0.9975025057792664 	 45	
4 0.9997246265411377 	 100	8.334808349609375 	 45	0.9976103901863098 	 47	
5 1.0020792484283447 	 100	1.8205680847167969 	 93	0.9974680542945862 	 40	
6 0.9981791973114014 	 100	1.4166252613067627 	 100	0.9975161552429199 	 45	
7 0.997626781463623 	 100	1.0559219121932983 	 71	0.9975817203521729 	 51	
8 1.0004053115844727 	 100	2.4647018909454346 	 15	0.997588574886322 	 33	
9 1.0083657503128052 	 100	1.141754388809204 	 85	0.9975098371505737 	 43	
10 1.0026594400405884 	 100	1.1343727111816406 	 100	0.9975334405899048 	 43	
11 0.9996399879455566 	 100	1.0926969051361084 	 100	0.9975703358650208 	 49	
12 1.0076016187667847 	 100	1.171639084815979 	 100	0.9974799752235413 	 48	
13 0.998628